In [ ]:
# Astronomy 400B
# Homework 7
# 
# Marco Barragan

In [ ]:
# Import modules
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib
import astropy.units as u
import astropy.table as tbl
from astropy.constants import G
from ReadFile import Read
from CenterofMass import CenterOfMass

In [ ]:
# Gravitational constant in units of kpc/Msun * (km/s)**2
G = G.to(u.kpc*u.km**2/u.s**2/u.Msun)
G

In [ ]:
class M33AnalyticOrbit:
    
    def __init__(filename):
        
        self.file = filename
        
        # Store the position and velocity COM for 31
        M31_COMP = M31COM.COM_P(0.1)
        M31_COMV = M31COM.COM_V(M31_COMP[0], M31_COMP[1], M31_COMP[2])

        # Store the position and velocity COM for M33
        M33_COMP = M33COM.COM_P(0.1)
        M33_COMV = M33COM.COM_V(M33_COMP[0], M33_COMP[1], M33_COMP[2])
        
        # Find the difference in the COMP and COMV between M33 and M31
        DeltaCOMP = M33_COMP - M31_COMP
        DeltaCOMV = M33_COMV - M31_COMV
        
        # Store the kinematic values of M33 relative to M33
        self.x = DeltaCOMP[0]
        self.y = DeltaCOMP[1]
        self.z = DeltaCOMP[2]
        self.vx = DeltaCOMV[0]
        self.vy = DeltaCOMV[1]
        self.vz = DeltaCOMV[2]
        
        # Store the mass values for each component of M31
        self.rDisk = 5 * u.kpc
        self.MDisk = 1.2e11 * u.Msol
        
        self.rBulge = 1 * u.kpc
        self.MBulge = 1.9e10 * u.Msol
        
        self.rHalo = 60 * u.kpc
        self.MHalo = 1.921e12 * u.Msol
        
    
    def HernquistAccel(self,M,r_a,x,y,z):
        r = sqrt(x**2 + y**2 + z**2)
        
        C = -G * M / (r(r_a + r)**2)
        
        ax = C*x
        ay = C*y
        az = C*z
        
        return np.array([ax,ay,az])
    
    def MiyamotoNagaiAccel(self,M,rd,x,y,z):
        zd = rd / 5
        R = sqrt(x**2 + y**2)
        B = rd + sqrt(z**2 + zd**2)
        
        C = -G * M /(R**2 + B**2)**1.5
        
        ax = C*x
        ay = C*y
        az = C*z * B / sqrt(z**2 + zd**2)
        
        return np.array([ax,ay,az])
    
    def M31Accel(self):
        a_disk = self.MiyamotoNagaiAccel(self.MDisk,self.rDisk,self.x,self.y,self.z)
        a_bulge = self.HernquistAccel(self.MBulge,self.rBulge,self.x,self.y,self.z)
        a_halo = self.HernquistAccel(self.MHalo,self.rHalo,self.x,self.y,self.z)
        
    def LeapFrog(self,dt,x,y,z,vx,vy,vz):
        r = 
        rhalf = 
        
        